<a href="https://colab.research.google.com/github/AssistMoli/LLaMA-Factory/blob/main/%E3%80%8CLLaMA_Factory_Tutorial_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
your_model = "Qwen/Qwen1.5-0.5B-Chat"

input_text = "被落石擊中！台鐵普悠瑪號445車次驚傳和平站出軌 大顆落石掉落軌道"

candidate_labels = ["詐騙"]


from transformers import pipeline



# Load zero-shot classification pipeline with GPT-2 model
classifier = pipeline("zero-shot-classification", model= your_model )

# Define input text and candidate labels

# Perform zero-shot classification
classification_result = classifier(input_text, candidate_labels)

# Display classification result
print("Input Text:", input_text)
print("Predicted Label:", classification_result["labels"][0])
print("Confidence Scores:", classification_result["scores"])






Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen1.5-0.5B-Chat and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Input Text: 被落石擊中！台鐵普悠瑪號445車次驚傳和平站出軌 大顆落石掉落軌道
Predicted Label: 詐騙
Confidence Scores: [0.04651827737689018]


## LLaMA Factory Colab Tutorial

Please use a **free** Tesla T4 Colab GPU to run this!

Project homepage: https://github.com/hiyouga/LLaMA-Factory

## Install Dependencies

In [ ]:
%rm -rf LLaMA-Factory
!git clone https://github.com/AssistMoli/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install .

## Check GPU environment

In [ ]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

## Log in with Hugging Face account to upload model (Optional)

In [ ]:
# !huggingface-cli login

## Fine-tune model via LLaMA Board

In [ ]:
# from llmtuner import create_ui
#
# create_ui().queue().launch(share=True)

## Customizing QA

In [ ]:
your_dataset = "fintech_qa"

In [ ]:
import pandas as pd

df = pd.read_excel(f'data/{your_dataset}.xlsx')
df = df.fillna("")

df.head()

In [ ]:
from json import loads, dumps

result = df.to_json(orient="records", force_ascii=False)
parsed = loads(result)
dumps(parsed)

In [ ]:
import json
with open(f"data/{your_dataset}.json", "w") as f:
   json.dump(parsed, f, indent=4,ensure_ascii=False)

## Parameters

In [ ]:
# 不知所云的中文模型 XD
your_model_name_or_path = "hfl/chinese-llama-2-1.3b"
your_template = "llama2"

your_dataset = "fintech_qa"

your_finetuning_type = "lora"
your_output_dir = "fintech_bot"
your_export_dir = "fintech_bot_export"
overwrite = True
your_fp16 = False
your_learning_rate = 0.000001
your_training_epoch = 1
your_max_samples = 500

In [ ]:
# 好用，但是會讓你想買　A100　的模型
your_model_name_or_path = "FlagAlpha/Llama2-Chinese-7b-Chat"
your_template = "llama2"

your_dataset = "fintech_qa"

your_finetuning_type = "lora"
your_output_dir = "fintech_bot"
your_export_dir = "fintech_bot_export"
overwrite = True
your_fp16 = False
your_learning_rate = 0.000001
your_training_epoch = 1
your_max_samples = 500

In [ ]:
# 原本作者 Qwen 的模型，還不錯用的輕量級中文模型，但簡體字居多，內容很多是中國大陸的內容
your_model_name_or_path = "Qwen/Qwen1.5-0.5B-Chat"
your_template = "qwen"

your_dataset = "fintech_qa"

your_finetuning_type = "lora"
your_output_dir = "fintech_bot"
your_export_dir = "fintech_bot_export"
overwrite = True
your_fp16 = True
your_learning_rate = 0.001
your_training_epoch = 100
your_max_samples = 5000



## Fine-tune model via Command Line

In [ ]:
from llmtuner import run_exp
run_exp(dict(
  stage="sft",
  do_train=True,
  model_name_or_path= your_model_name_or_path,
  dataset= your_dataset ,
  template=your_template,
  finetuning_type= your_finetuning_type ,
  lora_target="all",
  output_dir= your_output_dir ,
  per_device_train_batch_size=4,
  gradient_accumulation_steps=4,
  lr_scheduler_type="cosine",
  logging_steps=10,
  save_steps=100,
  learning_rate=your_learning_rate,
  num_train_epochs= your_training_epoch,
  max_samples=your_max_samples,
  max_grad_norm=1.0,
  fp16=your_fp16,
  overwrite_output_dir = overwrite
))



## Infer the fine-tuned model

In [ ]:
from llmtuner import ChatModel
chat_model = ChatModel(dict(
  model_name_or_path= your_model_name_or_path,
  adapter_name_or_path= your_output_dir,
  finetuning_type= your_finetuning_type,
  template= your_template,
))
messages = []
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    continue

  messages.append({"role": "user", "content": query})
  print("Assistant: ", end="", flush=True)
  response = ""
  for new_text in chat_model.stream_chat(messages):
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response})

## Merge LoRA weights

In [ ]:
from llmtuner import export_model
export_model(dict(
  model_name_or_path= your_model_name_or_path,
  adapter_name_or_path= your_output_dir,
  finetuning_type= your_finetuning_type,
  template= your_template,
  export_dir= your_export_dir ,
  # export_hub_model_id="your_hf_id/test_identity",
))